In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tensorflow
import tflearn
import json
import pickle

/Users/arunmainali/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
try: 
    with open('data.pickle', 'rb') as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data['intents']:
        for pattern in intent['patterns']:
            # print(pattern)
            wrds = nltk.word_tokenize(pattern)
            # print(wrds)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent['tag'])

        if intent['tag'] not in labels:
            labels.append(intent['tag'])
    words = [stemmer.stem(w.lower()) for w in words if w!='?']
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [4]:
with open('data.pickle', 'wb') as f:
    pickle.dump((words, labels, training, output), f)

In [5]:
tensorflow.compat.v1.reset_default_graph()
net = tflearn.input_data(shape = [None, len(training[0])])
net = tflearn.fully_connected(net, 12)
net = tflearn.fully_connected(net, 12)
net = tflearn.fully_connected(net, len(output[0]), activation = 'softmax')
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
model = tflearn.DNN(net)
model.fit(training, output, n_epoch = 1800, batch_size =12, show_metric = True)

Training Step: 19799  | total loss: 0.03743 | time: 0.006s
| Adam | epoch: 1800 | loss: 0.03743 - acc: 0.9990 -- iter: 120/127
Training Step: 19800  | total loss: 0.03590 | time: 0.006s
| Adam | epoch: 1800 | loss: 0.03590 - acc: 0.9991 -- iter: 127/127
--


In [9]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words: 
        for i, w in enumerate(words):
            if w == se:
                 bag[i] = 1
    return numpy.array(bag)

In [ ]:
import random
def chat():
    print('Start talking with BOT of Texas College(TexasAIChatBOT)')
    while True:
        inp = input('You:')
        print('')
        if inp.lower() == 'quit':
            break
        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        # print(tag)
        # print(results)

        if results[results_index] > 0.4:
            for tg in data['intents']:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
            print('')
            print('')
        else:
            print('I dont understand your question, try again')
            print('')
chat()

Start talking with BOT of Texas College(TexasAIChatBOT)


You: hello



Hey




You: who are you



I am the interactive agent of Texas College




In [ ]:
!pip3 install gradio == 4.44.1      #gradio provies the server for free, or helps to generate the public url 

In [ ]:
import gradio
def find_sq(inp):
    return int(inp)**2
demo = gradio.Interface(fn = find_sq, inputs = 'text', outputs = 'text')
demo.launch(share = True)

In [ ]:
import gradio
def chat(Chatbot_by_Rojan_Upreti):

    results = model.predict([bag_of_words(Chatbot_by_Rojan_Upreti, words)])[0]
    results_index = numpy.argmax(results)
    tag = labels[results_index]

    if results[results_index] > 0.4:
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
        return random.choice(responses)

    else:
        return 'I dont understand your question, try again'

demo = gradio.Interface(fn = chat, inputs  = 'text', outputs = 'text')
demo.launch(share = True)

In [ ]:
def find_sq(inp):
    return inp**2
find_sq(4)

In [ ]:
def greet(name):
    return "Hello " + name
print(greet("ram"))

In [ ]:
import gradio as gr

def greet(name):
    return "Hello " + name

demo = gr.Interface(fn=greet, inputs="textbox", outputs="textbox")
    
demo.launch(share=True)